<a href="https://colab.research.google.com/github/Vnyk121098/Solubility/blob/main/solubility(DL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving curated-solubility-dataset.csv to curated-solubility-dataset (1).csv


In [ ]:
import pandas as pd
sf = pd.read_csv('curated-solubility-dataset.csv')
df = sf.fillna({'MolWt':sf.mean(axis=0)['MolWt'],'MolLogP':sf.mean(axis=0)['MolLogP'],'MolMR':sf.mean(axis=0)['MolMR'],'HeavyAtomCount':sf.mean(axis=0)['HeavyAtomCount'],'NumHAcceptors':sf.mean(axis=0)['NumHAcceptors'],'NumHDonors':sf.mean(axis=0)['NumHDonors'],'NumHeteroatoms':sf.mean(axis=0)['NumHeteroatoms'],'NumRotatableBonds':sf.mean(axis=0)['NumRotatableBonds'],'NumValenceElectrons':sf.mean(axis=0)['NumValenceElectrons'],'NumAromaticRings':sf.mean(axis=0)['NumAromaticRings'],'NumSaturatedRings':sf.mean(axis=0)['NumSaturatedRings'],'NumAliphaticRings':sf.mean(axis=0)['NumAliphaticRings'],'RingCount':sf.mean(axis=0)['RingCount'],'TPSA':sf.mean(axis=0)['TPSA'],'LabuteASA':sf.mean(axis=0)['LabuteASA'],'BalabanJ':sf.mean(axis=0)['BalabanJ'],'BertzCT':sf.mean(axis=0)['BertzCT'],'Solubility':sf.mean(axis=0)['Solubility']})
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for i in ['MolWt','MolLogP','MolMR','HeavyAtomCount','NumHAcceptors','NumHDonors','NumHeteroatoms','NumRotatableBonds','NumValenceElectrons','NumAromaticRings','NumSaturatedRings','NumAliphaticRings','RingCount','TPSA','LabuteASA','BalabanJ','BertzCT','Solubility']:
  scaler.fit(df[[i]])
  df[i] = scaler.transform(df[[i]])
print(df.head(5))
from IPython.display import display
display(df)

    ID  ...   BertzCT
0  A-3  ...  0.010153
1  A-4  ...  0.024673
2  A-5  ...  0.009781
3  A-8  ...  0.094818
4  A-9  ...  0.037157

[5 rows x 26 columns]


,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,MolLogP,MolMR,HeavyAtomCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,0.624169,0.000000,1,G1,0.072489,0.409739,0.072178,0.056848,0.000000,0.000000,0.022472,0.120567,0.070577,0.000000,0.000000,0.000000,0.000000,0.000000,0.067928,5.593747e-07,0.010153
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,0.647773,0.000000,1,G1,0.030276,0.395549,0.036567,0.031008,0.011628,0.038462,0.022472,0.000000,0.030815,0.057143,0.000000,0.033333,0.083333,0.023964,0.030443,3.436069e-01,0.024673
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,0.718166,0.000000,1,G1,0.024867,0.393236,0.025955,0.020672,0.011628,0.000000,0.022472,0.007092,0.022863,0.028571,0.000000,0.000000,0.027778,0.014057,0.022831,4.003805e-01,0.009781
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,0.604033,0.000000,1,G1,0.141238,0.447741,0.141410,0.134367,0.069767,0.076923,0.078652,0.070922,0.131213,0.171429,0.000000,0.000000,0.166667,0.099412,0.142252,5.902762e-07,0.094818
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,0.555850,0.000000,1,G1,0.078162,0.396279,0.083895,0.077519,0.069767,0.000000,0.067416,0.085106,0.081511,0.057143,0.133333,0.133333,0.166667,0.046610,0.079021,1.442578e-01,0.037157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9977,I-84,tetracaine,InChI=1S/C15H24N2O2/c1-4-5-10-16-14-8-6-13(7-9...,GKCBAIGFKIBETG-UHFFFAOYSA-N,C(c1ccc(cc1)NCCCC)(=O)OCCN(C)C,0.663761,0.000000,1,G1,0.048268,0.397482,0.055431,0.046512,0.046512,0.038462,0.044944,0.056738,0.052684,0.028571,0.000000,0.000000,0.027778,0.034233,0.048487,3.185383e-01,0.018061
9978,I-85,tetracycline,InChI=1S/C22H24N2O8/c1-21(31)8-5-4-6-11(25)12(...,OFVLGDICTFRJMM-WESIUVDSSA-N,OC1=C(C(C2=C(O)[C@@](C(C(C(N)=O)=C(O)[C@H]3N(C...,0.668986,0.000000,1,G1,0.082305,0.371604,0.077177,0.080103,0.104651,0.230769,0.112360,0.014184,0.084493,0.028571,0.000000,0.100000,0.111111,0.149563,0.078682,2.724279e-01,0.055433
9979,I-86,thymol,InChI=1S/C10H14O/c1-7(2)9-5-4-8(3)6-10(9)11/h4...,MGSRCZKZVOBKFT-UHFFFAOYSA-N,c1(cc(ccc1C(C)C)C)O,0.717322,0.019222,3,G5,0.026691,0.399374,0.033066,0.025840,0.011628,0.038462,0.011236,0.007092,0.029821,0.028571,0.000000,0.000000,0.027778,0.016659,0.027070,4.114134e-01,0.012116
9980,I-93,verapamil,"InChI=1S/C27H38N2O4/c1-20(2)27(19-28,22-10-12-...",SGTNSNPWRIOYBX-UHFFFAOYSA-N,COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,0.600402,0.000000,1,G1,0.084227,0.420112,0.092758,0.082687,0.069767,0.000000,0.067416,0.092199,0.089463,0.057143,0.000000,0.000000,0.055556,0.052662,0.085942,2.691570e-01,0.045280


In [ ]:
parameters = ['MolWt','MolLogP','MolMR','HeavyAtomCount','NumHAcceptors','NumHDonors','NumHeteroatoms','NumRotatableBonds','NumValenceElectrons','NumAromaticRings','NumSaturatedRings','NumAliphaticRings','RingCount','TPSA','LabuteASA','BalabanJ','BertzCT']
nf = df[parameters]
from IPython.display import display
#display(nf.head(5))
inf = nf[parameters]
outf = df['Solubility']
#print(outf)
print(outf.shape,inf.shape)

(9982,) (9982, 17)


In [ ]:
import tensorflow as tf
#tf.enable_eager_execution()

training_df: pd.DataFrame = pd.concat([inf,outf],axis=1)
features = [parameters]
#print(training_df)

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(training_df[parameters].values, tf.float32),
            tf.cast(training_df['Solubility'].values, tf.float32)
        )
    )
)

for parameters_tensor, Solubility_tensor in training_dataset:
    f'parameters:{parameters_tensor} target:{Solubility_tensor}'
print(parameters_tensor.shape,Solubility_tensor.shape)

(18,) ()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(parameters_tensor,Solubility_tensor,train_size=0.8)

TypeError: ignored

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

model = Sequential()
model.add(Flatten(input_shape=[1, 17]))
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 169694)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               16969500  
Total params: 16,969,500
Trainable params: 16,969,500
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])
model.fit(parameters_tensor,Solubility_tensor)

IndexError: ignored